In [38]:
import glob
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader , random_split
from torchsummary import summary
import matplotlib.pyplot as plt
from random import randint
from tqdm import tqdm
from tqdm.notebook import trange, tqdm
import tensorflow as tf
import keras
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score


In [3]:
print(tf.__version__)

2.18.0


In [11]:
# Dataloader foe FER training data

class FerTrain(Dataset):
    def __init__(self): # add additional parameters needed to load the dataset e.g dataset path
        self.train_images  = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\train_images.npy')
        self.train_labels = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\train_labels.npy')
        len(self.train_images)


    def __len__(self):
        return len(self.train_images)

    def __getitem__(self, idx):
        # your code here
        img = self.train_images[idx,:]
        img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_LINEAR)
        img= img.reshape(256,256,1)
        img=img.astype(np.float32)
        img=img.T
        img_tensor = torch.from_numpy(img)

        label= torch.from_numpy(self.train_labels[idx]).to(torch.float32)

        return img_tensor, label

In [12]:
# Dataloader foe FER testing data

class FerTest(Dataset):
  def __init__(self): # add additional parameters needed to load the dataset e.g dataset path
        self.test_images  = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\test_images.npy')
        self.test_labels = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\test_labels.npy')

  def __len__(self):
        return self.test_images.shape[0]

  def __getitem__(self, idx):
      # your code here
      img = self.test_images[idx]
      img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_LANCZOS4)
      img= img.reshape(256,256,1)
      img=img.astype(np.float32)
      img=img.T
      img_tensor = torch.from_numpy(img)

      label= torch.from_numpy(self.test_labels[idx]).to(torch.float32)

      return img_tensor, label

In [13]:
fer = FerTrain()
fer.__getitem__(6708)


(tensor([[[0.2784, 0.2784, 0.2784,  ..., 0.7137, 0.7137, 0.7137],
          [0.2784, 0.2784, 0.2784,  ..., 0.7137, 0.7137, 0.7137],
          [0.2784, 0.2784, 0.2784,  ..., 0.7137, 0.7137, 0.7137],
          ...,
          [0.0627, 0.0627, 0.0627,  ..., 0.5098, 0.5098, 0.5098],
          [0.0627, 0.0627, 0.0627,  ..., 0.5098, 0.5098, 0.5098],
          [0.0627, 0.0627, 0.0627,  ..., 0.5098, 0.5098, 0.5098]]]),
 tensor([0., 1., 0.]))

In [14]:
class EmotionDetectionNet(nn.Module):
    def __init__(self):
        super(EmotionDetectionNet, self).__init__()

        # Convolutional Layers
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),  # 255x255 -> 255x255
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),  # 255x255 -> 128x128
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # 128x128 -> 64x64
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1),  # 64x64 -> 32x32
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # 32x32 -> 16x16
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),  # 16x16 -> 8x8
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

        # Fully Connected Layers
        self.fc = nn.Sequential(
            nn.Linear(256 * 8 * 8, 512),  # Flattened feature size -> 512
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),  # Reduce to 256 neurons
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 3),  # 3 classes for classification
        )

    def forward(self, x):
        # Pass through convolutional layers
        x = self.conv(x)
        # Flatten the tensor for fully connected layers
        x = torch.flatten(x, start_dim=1)
        # Pass through fully connected layers
        x = self.fc(x)
        return x


In [15]:
device=None
if torch.cuda.is_available():
  device = 'cuda:0'
else:
  device = 'cpu'
print(device)
# Initialize model
model = EmotionDetectionNet()
# Display the model summary
# summary(model, (1, 48, 48))
net =  EmotionDetectionNet()
model.to(device)
# net.to(device)
summary(model,(1,256,256))


cuda:0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]             640
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 128, 128]          36,928
       BatchNorm2d-5         [-1, 64, 128, 128]             128
              ReLU-6         [-1, 64, 128, 128]               0
            Conv2d-7          [-1, 128, 64, 64]          73,856
       BatchNorm2d-8          [-1, 128, 64, 64]             256
              ReLU-9          [-1, 128, 64, 64]               0
           Conv2d-10          [-1, 128, 32, 32]         147,584
      BatchNorm2d-11          [-1, 128, 32, 32]             256
             ReLU-12          [-1, 128, 32, 32]               0
           Conv2d-13          [-1, 256, 16, 16]         295,168
      BatchNorm2d-14          [-

In [25]:
bath_size = 32
print(len(fer))
split_size_train = int(len(fer)*0.75)
split_size_val = len(fer) - split_size_train
train,valid = random_split(fer, lengths=[split_size_train, split_size_val])
print(len(train),len(valid))
train_dataloader = DataLoader(train, batch_size=bath_size, shuffle=True)
valid_dataloader = DataLoader(valid, batch_size=bath_size, shuffle=True)


16175
12131 4044


In [26]:
learning_rate = 1e-4
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
def calculate_accuracy(predictions, labels):
    predicted = torch.argmax(predictions, dim=1)
    orig = torch.argmax(labels, dim=1)
    correct = (predicted == orig).sum().item()
    return correct

In [29]:
#training loop

last_acc= 100
print(device)
for param in model.parameters():
    param.requires_grad = True
for epoch in tqdm(range(50)):
    # monitor training loss
    train_loss = 0.0
    model.train()
    train_accuracy = 0.0
    #Training

    for data,label in tqdm(train_dataloader):

        images = data
        images = images.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        label=label.to(device)

        train_accuracy += calculate_accuracy(outputs, label)
        loss = loss_function(outputs, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()


    valid_loss = 0.0
    val_accuracy = 0.0
    model.eval()
    for data, label in valid_dataloader:

        data = data.to(device)
        target = model(data)
        label=label.to(device)
        val_accuracy += calculate_accuracy(target, label)
        loss = loss_function(target, label)
        valid_loss += loss.item()

    #Save the best model weights
    
    if val_accuracy/len(valid) < last_acc:
        last_acc = val_accuracy
        torch.save(model.state_dict(),'C:\\Users\\tld\\Downloads\\cs5805project\\weights\\FER_3_256x256_3.pt')

    print("EPOCH ", epoch+1,"   Training acc ",train_accuracy/len(train), "validation acc = " ,val_accuracy/len(valid))

cuda:0


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/380 [00:00<?, ?it/s]

EPOCH  1    Training acc  0.6784271700601764 validation acc =  0.6822453016815034


  0%|          | 0/380 [00:00<?, ?it/s]

...

EPOCH  48    Training acc  0.9953837276399308 validation acc =  0.7161226508407518


  0%|          | 0/380 [00:00<?, ?it/s]

EPOCH  49    Training acc  0.994064792679911 validation acc =  0.7111770524233432


  0%|          | 0/380 [00:00<?, ?it/s]

EPOCH  50    Training acc  0.9934053251999011 validation acc =  0.7223046488625123


In [30]:
model_test = EmotionDetectionNet()
model_test.load_state_dict(torch.load('C:\\Users\\tld\\Downloads\\cs5805project\\weights\\FER_3_256x256_3.pt'))
model_test.to(device)
fer_test = FerTest()
test_dataloader = DataLoader(fer_test, batch_size=25, shuffle=False)
test_accuracy = 0.0
print(len(fer_test))
for data, label in test_dataloader:
  data = data.to(device)
  target = model_test(data)
  label=label.to(device)
  test_accuracy += calculate_accuracy(target, label)

print(f'Test Accuracy: {test_accuracy/len(fer_test)}')

C:\Users\tld\AppData\Local\Temp\ipykernel_8476\1179852314.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_test.load_state_dict(torch.load('C:\\Users\\tld\\Download

3965
Test Accuracy: 0.6895334174022698


## Fine Tune on the MLI-DER dataset

In [31]:
class DrivingFineTune(Dataset):
    def __init__(self, samples=100): # add additional parameters needed to load the dataset e.g dataset path
        self.train_images  = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\train_images_fine_tune.npy')
        random_samples = np.random.randint(self.train_images.shape[0], size=samples)
        self.train_images = self.train_images[random_samples, :]
        self.train_labels = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\train_labels_fine_tune.npy')
        self.train_labels = self.train_labels[random_samples,:]
        len(self.train_images.shape)

    def __len__(self):
        return len(self.train_images)

    def __getitem__(self, idx):
        img = self.train_images[idx]
        # img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_LINEAR)
        # img= img.reshape(256,256,1)
        img=img.astype(np.float32)
        img=img.T
        img_tensor = torch.from_numpy(img)

        label= torch.from_numpy(self.train_labels[idx]).to(torch.float32)

        return img_tensor, label

In [32]:
driving = DrivingFineTune()
driving.__getitem__(50)

(tensor([[[0.5690, 0.3975, 0.2829,  ..., 0.3730, 0.3783, 0.3613],
          [0.8153, 0.7934, 0.8039,  ..., 0.3542, 0.3751, 0.3787],
          [0.7829, 0.7969, 0.8036,  ..., 0.3788, 0.4069, 0.3637],
          ...,
          [0.7857, 0.4875, 0.4476,  ..., 0.3533, 0.3395, 0.3382],
          [0.8466, 0.6264, 0.4134,  ..., 0.3137, 0.3240, 0.3113],
          [0.8356, 0.6870, 0.3474,  ..., 0.2967, 0.2950, 0.2942]]]),
 tensor([1., 0., 0.]))

In [34]:
model_fine_tune = EmotionDetectionNet()

for samples in range(100,801, 100):
    model_fine_tune.load_state_dict(torch.load('C:\\Users\\tld\\Downloads\\cs5805project\\weights\\FER_3_256x256_3.pt', weights_only=True))
    learning_rate = 1e-4
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model_fine_tune.to(device)
    fer_fine_tune = DrivingFineTune(samples=samples)
    fine_tune_dataloader = DataLoader(fer_fine_tune, batch_size=10, shuffle=True)
    for param in model_fine_tune.parameters():
        param.requires_grad = True


    for epoch in tqdm(range(30)):
        train_loss = 0.0
        model_fine_tune.train()
        train_accuracy = 0.0
        #Training

        for data,label in fine_tune_dataloader:

            images = data
            images = images.to(device)
            optimizer.zero_grad()
            outputs = model_fine_tune(images)

            label=label.to(device)

            train_accuracy += calculate_accuracy(outputs, label)
            loss = loss_function(outputs, label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
    print(f"Samples: {samples} EPOCH: ", epoch+1,"   Training acc ",train_accuracy/len(fer_fine_tune))
    torch.save(model.state_dict(),f'C:\\Users\\tld\\Downloads\\cs5805project\\Base Model\\Base2\\base_{samples}.pt')
    

  0%|          | 0/30 [00:00<?, ?it/s]

Samples: 100 EPOCH:  30    Training acc  0.22


  0%|          | 0/30 [00:00<?, ?it/s]

Samples: 200 EPOCH:  30    Training acc  0.355


  0%|          | 0/30 [00:00<?, ?it/s]

...

Samples: 600 EPOCH:  30    Training acc  0.2683333333333333


  0%|          | 0/30 [00:00<?, ?it/s]

Samples: 700 EPOCH:  30    Training acc  0.29


  0%|          | 0/30 [00:00<?, ?it/s]

Samples: 800 EPOCH:  30    Training acc  0.31


## Test Performance on the MLI-DER test set

In [36]:
class FerDrivingTest(Dataset):
  def __init__(self): # add additional parameters needed to load the dataset e.g dataset path
        self.test_images  = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\driving_data_test_images.npy')
        self.test_labels = np.load('C:\\Users\\tld\\Downloads\\cs5805project\\driving_data_test_labels.npy')
        print(self.test_images.shape)

  def __len__(self):
        return self.test_images.shape[0]

  def __getitem__(self, idx):
      # your code here
      img = self.test_images[idx]
    #   img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_LANCZOS4)
    #   img= img.reshape(256,256,1)
      img=img.astype(np.float32)
      img=img.T
      img_tensor = torch.from_numpy(img)
      label= torch.from_numpy(self.test_labels[idx]).to(torch.float32)

      return img_tensor, label

In [39]:
model_test = EmotionDetectionNet()
for models in glob.glob('C:\\Users\\tld\\Downloads\\cs5805project\\Base Model\\Base2\\*.pt'):
    num_samples = models.split('_')[-1][:3]
    model_test.load_state_dict(torch.load(models, weights_only=True))
    model_test.to(device)
    fer_test = FerDrivingTest()
    test_dataloader = DataLoader(fer_test, batch_size=1, shuffle=False)
    preds = []
    labels_arg_max = []
    
    for data, label in test_dataloader:
        data = data.to(device)
        target = model_test(data)
        label=label.to(device)
        label_max = torch.argmax(label)
        pred = torch.argmax(target)
        preds.append(label_max.item())
        labels_arg_max.append(pred.item())

    print(f'Samples: {num_samples} Test Accuracy: {accuracy_score(preds, labels_arg_max)} F1 score: {f1_score(preds, labels_arg_max, average='weighted')}')

(938, 256, 256, 1)
Samples: 100 Test Accuracy: 0.31236673773987206 F1 score: 0.30510148726561664
(938, 256, 256, 1)
Samples: 200 Test Accuracy: 0.3017057569296375 F1 score: 0.29412901196652047
(938, 256, 256, 1)
Samples: 300 Test Accuracy: 0.3091684434968017 F1 score: 0.3025167573298959
(938, 256, 256, 1)
Samples: 400 Test Accuracy: 0.31663113006396587 F1 score: 0.30806601868756095
(938, 256, 256, 1)
Samples: 500 Test Accuracy: 0.31982942430703626 F1 score: 0.3127851809727359
(938, 256, 256, 1)
Samples: 600 Test Accuracy: 0.31663113006396587 F1 score: 0.3098274279978116
(938, 256, 256, 1)
Samples: 700 Test Accuracy: 0.31023454157782515 F1 score: 0.3033743084796602
(938, 256, 256, 1)
Samples: 800 Test Accuracy: 0.3230277185501066 F1 score: 0.31448463643786423
